In [ ]:
# default_exp signUp

# SignUp

> Interface to sign up user

In [31]:
#hide
from nbdev.showdoc import *
import src.setEnv 

In [32]:
#export 
from src.userTable import UserTable
from awsSchema.apigateway import Response, Event
from copy import deepcopy
from beartype import beartype
from uuid import uuid4
from src.saltHashPassword import hash_password, check_password
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from time import time

In [33]:
#check whether hash password is working here
examplePassword1 = '123'
exampleHashedPassword1 = hash_password(examplePassword1)
exampleHashedPassword1

'86daa96273f4957fcbfae83d4515c960436dec2fe9949c1ef787b6b2316bb6cd:e6f52be8e683430aa6a4ccb8ba4d4dcb'

## User Input Class

In [34]:
#export 
@dataclass_json
@dataclass
class UserInput:
    username: str
    password: str
    
    @property
    def passwordHash(self):
        return hash_password(self.password)
    
    def saveTable(self):
        passwordHashed = self.passwordHash
        userTable = UserTable(
                userId = str(uuid4()),
                username = self.username,
                date = time(),
                passwordHash = passwordHashed
            )
        userTable.save()
        

In [35]:
example = UserInput.from_dict({'username': 'DelG', 'password': '123'})
example

UserInput(username='DelG', password='123')

In [36]:
#export 
class H:
    class ParseInputError(Exception): pass
    class SavingError(Exception): pass
    @staticmethod
    @beartype
    def parseInput(event:dict)->UserInput:
        try:
            user = Event.parseDataClass(UserInput, deepcopy(event))
            return user
        except Exception as e:
            raise ParseInputError(e)
    
    @classmethod
    @beartype
    def saveUserMethod(cls, user:UserInput)->bool:
        try:
            user.saveTable()
            return True
        except Exception as e:
            raise cls.SavingError(e)
    
        

## Main Lambda Handler

In [37]:
#export
def signUp(events, *args):
    try:
        user = H.parseInput(event)
        H.saveUserMethod(user)
        return Response.returnSuccess()
    except H.SavingError as e:
        return Response.returnError(f'failed saving user {e}')
    except Exception as e:
        return Response.returnError(f' unknown error {e}')
    

In [38]:
input_ = {'username': 'DelG', 'password': '123'}
event = Event.getInput(body = input_)
signUp(event)

{'body': '{"error":"failed saving user As of v1.0 PyanmoDB Models must have a table_name\\nModel: src.userTable.UserTable\\nSee https:\\/\\/pynamodb.readthedocs.io\\/en\\/latest\\/release_notes.html"}',
 'statusCode': 400,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

In [37]:
epoch = time()
epoch

1658677547.060287